In [ ]:
import os
from pathlib import Path
import polars as pl
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import kaggle_evaluation.mcts_inference_server

# Constants
COMP_PATH = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants')
TARGET = 'utility_agent1'

# Helper functions
def rmse(y_true, y_pred):
    """Calculate Root Mean Square Error (RMSE)."""
    return sqrt(mean_squared_error(y_true, y_pred))

class DataPreprocessor:
    """Class for handling data preprocessing tasks."""
    
    def __init__(self, train_data_path, target_col):
        self.train_data_path = train_data_path
        self.target_col = target_col
        self.obj_cols = None
        self.encoder = OrdinalEncoder(handle_unknown='use_encoded_value', 
                                      unknown_value=-999, 
                                      encoded_missing_value=-9999)
        
    def load_data(self):
        """Load and preprocess the data."""
        train = pl.read_csv(self.train_data_path)
        y_train = train[self.target_col]
        cols_to_drop = ['num_draws_agent1', 'num_losses_agent1', 'num_wins_agent1', self.target_col]
        train = train.drop(cols_to_drop)
        self.obj_cols = train.select(pl.col(pl.String)).columns
        return train, y_train
    
    def encode_data(self, data):
        """Apply ordinal encoding to categorical columns."""
        transformed = self.encoder.transform(data[self.obj_cols])
        for idx, col in enumerate(self.obj_cols):
            data = data.with_columns(pl.Series(col, transformed[:, idx]))
        return data
    
    def fit_encoder(self, data):
        """Fit the ordinal encoder on training data."""
        self.encoder.fit(data[self.obj_cols])

class ModelTrainer:
    """Class to handle training and evaluation of models."""
    
    def __init__(self, models, X_train, y_train, X_test, y_test):
        self.models = models
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.best_model = None
        self.rmse_scores = {}
        
    def train_and_evaluate(self):
        """Train and evaluate models, then select the best one based on RMSE."""
        for model_name, model in self.models.items():
            model.fit(self.X_train, self.y_train)
            y_pred = model.predict(self.X_test)
            self.rmse_scores[model_name] = rmse(self.y_test, y_pred)
            print(f'{model_name} RMSE: {self.rmse_scores[model_name]:.4f}')
        
        # Select the best model based on RMSE
        best_model_name = min(self.rmse_scores, key=self.rmse_scores.get)
        self.best_model = self.models[best_model_name]
        print(f'Best model: {best_model_name} with RMSE: {self.rmse_scores[best_model_name]:.4f}')
        return self.best_model
    
class PredictionService:
    """Class for handling model inference and prediction."""
    
    def __init__(self, preprocessor, best_model, target_col):
        self.preprocessor = preprocessor
        self.best_model = best_model
        self.target_col = target_col
        self.counter = 0
    
    def predict(self, test, submission):
        """Predict on test data and prepare submission."""
        self.counter += 1
        test_transformed = self.preprocessor.encode_data(test)
        predictions = self.best_model.predict(test_transformed)
        return submission.with_columns(pl.Series(self.target_col, predictions))

def run_inference_server(predict_function):
    """Run inference server for submission or local testing."""
    inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict_function)
    
    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        inference_server.serve()
    else:
        inference_server.run_local_gateway(
            (
                '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
                '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
            )
        )

def main():
    """Main entry point of the script."""
    
    # Initialize data preprocessor and load data
    preprocessor = DataPreprocessor(COMP_PATH / 'train.csv', TARGET)
    train_data, y_train = preprocessor.load_data()
    preprocessor.fit_encoder(train_data)
    train_data_encoded = preprocessor.encode_data(train_data)
    
    # Split data into training and testing sets
    X_train, X_test, y_train_split, y_test_split = train_test_split(train_data_encoded, y_train, test_size=0.2, random_state=42)
    
    # Define models
    models = {
        "RandomForest": RandomForestRegressor(n_estimators=100, max_depth=5, n_jobs=-1),
        "ExtraTrees": ExtraTreesRegressor(n_estimators=100, max_depth=5, n_jobs=-1),
        "XGBoost": xgb.XGBRegressor(n_estimators=100, max_depth=5, n_jobs=-1),
        "LightGBM": lgb.LGBMRegressor(n_estimators=100, max_depth=5, n_jobs=-1)
    }
    
    # Train models and select the best one
    model_trainer = ModelTrainer(models, X_train, y_train_split, X_test, y_test_split)
    best_model = model_trainer.train_and_evaluate()
    
    # Start the prediction service
    prediction_service = PredictionService(preprocessor, best_model, TARGET)
    
    # Run inference server for predictions
    run_inference_server(prediction_service.predict)

if __name__ == "__main__":
    main()
